In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,roc_auc_score,f1_score,precision_score,recall_score,roc_curve,precision_recall_curve, classification_report
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Copy the data file to current working directory
!cp '/content/drive/MyDrive/Colab Notebooks/ICAIML Project/data.csv' ./

In [4]:
df = pd.read_csv("data.csv")

In [5]:
df

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


In [10]:
df_X=df.drop(labels=['Bankrupt?'], axis=1)
df_y=df['Bankrupt?']

scaler=StandardScaler()
df_X=scaler.fit_transform(df_X)

In [11]:
train_X, test_X, train_y, test_y = train_test_split(df_X, df_y,test_size=0.2, stratify = df["Bankrupt?"])

In [12]:
print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Shapes:  (5455, 95) (5455,) (1364, 95) (1364,)


**Default Logistic Regression:**

In [13]:
lr_model=LogisticRegression()
lr_model.fit(train_X, train_y)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
y_pred=lr_model.predict(test_X)

In [18]:
print("Default LR model metrics:")
print("Classification Report:\n", classification_report(test_y, y_pred))
print("Accuracy Score:\n", accuracy_score(test_y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_y, y_pred))

Default LR model metrics:
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      1320
           1       0.39      0.16      0.23        44

    accuracy                           0.96      1364
   macro avg       0.68      0.58      0.60      1364
weighted avg       0.95      0.96      0.96      1364

Accuracy Score:
 0.9648093841642229
Confusion Matrix:
 [[1309   11]
 [  37    7]]


**Resampling**

In [45]:
df_non_bankrupt = df[(df["Bankrupt?"]==0)] 
df_bankrupt = df[(df["Bankrupt?"]==1)] 
df_bankrupt_resampled = resample(df_bankrupt, replace=True, n_samples= len(df_non_bankrupt), random_state=42)
df_resampled = pd.concat([df_bankrupt_resampled, df_non_bankrupt])

In [44]:
df_resampled["Bankrupt?"].value_counts()

1    6599
0    6599
Name: Bankrupt?, dtype: int64

In [46]:
scaler=StandardScaler()
X_scale=scaler.fit_transform(df_resampled.drop(labels=['Bankrupt?'], axis=1))

In [47]:
train_resample_X, test_resample_X, train_resample_y, test_resample_y = train_test_split(X_scale, df_resampled["Bankrupt?"],test_size=0.3)

In [48]:
lr_resample_model=LogisticRegression()
lr_resample_model.fit(train_resample_X,train_resample_y)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [49]:
y_pred=lr_resample_model.predict(test_resample_X)

In [50]:
print("Resampling data model metrics:")
print("Classification Report:\n", classification_report(test_resample_y, y_pred))
print("Accuracy Score:\n", accuracy_score(test_resample_y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_resample_y, y_pred))

Resampling data model metrics:
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.87      0.88      2001
           1       0.87      0.89      0.88      1959

    accuracy                           0.88      3960
   macro avg       0.88      0.88      0.88      3960
weighted avg       0.88      0.88      0.88      3960

Accuracy Score:
 0.8795454545454545
Confusion Matrix:
 [[1742  259]
 [ 218 1741]]


**Balanced Bagging Classifier**

In [19]:
bb_model = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42)
bb_model.fit(train_X, train_y)

/usr/local/lib/python3.9/dist-packages/imblearn/ensemble/_bagging.py:353: FutureWarning: `base_estimator` was renamed to `estimator` in version 0.10 and will be removed in 0.12.
  warnings.warn(


BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                          random_state=42, sampling_strategy='not majority')

In [30]:
y_pred=bb_model.predict(test_X)

In [31]:
print("Balanced Bagging Classifier model metrics:")
print("Classification Report:\n", classification_report(test_y, y_pred))
print("Accuracy Score:\n", accuracy_score(test_y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_y, y_pred))

Balanced Bagging Classifier model metrics:
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1320
           1       0.40      0.09      0.15        44

    accuracy                           0.97      1364
   macro avg       0.69      0.54      0.57      1364
weighted avg       0.95      0.97      0.96      1364

Accuracy Score:
 0.966275659824047
Confusion Matrix:
 [[1314    6]
 [  40    4]]


**SMOTE on train data**

In [24]:
oversample = SMOTE()
train_X_oversample, train_y_oversample = oversample.fit_resample(train_X, train_y)

In [28]:
train_y_oversample.value_counts()

0    5279
1    5279
Name: Bankrupt?, dtype: int64

In [25]:
lr_smote_train_model=LogisticRegression()
lr_smote_train_model.fit(train_X_oversample, train_y_oversample)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
y_pred=lr_smote_train_model.predict(test_X)

In [33]:
print("SMOTE on train data model metrics:")
print("Classification Report:\n", classification_report(test_y, y_pred))
print("Accuracy Score:\n", accuracy_score(test_y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_y, y_pred))

SMOTE on train data model metrics:
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.94      1320
           1       0.18      0.73      0.29        44

    accuracy                           0.88      1364
   macro avg       0.58      0.81      0.61      1364
weighted avg       0.96      0.88      0.92      1364

Accuracy Score:
 0.8826979472140762
Confusion Matrix:
 [[1172  148]
 [  12   32]]


**SMOTE** on entire data

In [34]:
X=df.drop(labels=['Bankrupt?'], axis=1)
y=df['Bankrupt?']

In [35]:
oversample = SMOTE()
X,y=oversample.fit_resample(X,y)

In [36]:
scaler=StandardScaler()
X_scale=scaler.fit_transform(X)

In [37]:
train_smote_full_X, test_smote_full_X, train_smote_full_y, test_smote_full_y = train_test_split(X_scale, y,test_size=0.3)

In [38]:
lr_smote_full_model=LogisticRegression()
lr_smote_full_model.fit(train_smote_full_X,train_smote_full_y)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
y_pred=lr_smote_full_model.predict(test_smote_full_X)

In [41]:
print("SMOTE on full data model metrics:")
print("Classification Report:\n", classification_report(test_smote_full_y, y_pred))
print("Accuracy Score:\n", accuracy_score(test_smote_full_y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_smote_full_y, y_pred))

SMOTE on full data model metrics:
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91      2008
           1       0.89      0.93      0.91      1952

    accuracy                           0.91      3960
   macro avg       0.91      0.91      0.91      3960
weighted avg       0.91      0.91      0.91      3960

Accuracy Score:
 0.9065656565656566
Confusion Matrix:
 [[1778  230]
 [ 140 1812]]
